This code was used in experimenting with this as a supervised learning problem rather than unsupervised learning. I wanted to see if after training on a few mountains, the ability level of Winter Park cuold be predicted. Comb_tables has changed since this notebook, so it will not run anymore.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
%matplotlib inline

# DO NOT RUN COMB_TABLES AGAIN

In [2]:
run comb_tables.py

In [3]:
whole_table['ability_level'][whole_table['ability_level'] == 'Advanced Intermediate'] = 'Advanced'
whole_table['ability_level'][whole_table['ability_level'] == 'Adv. Intermediate'] = 'Advanced'
whole_table['ability_level'][whole_table['ability_level'] == 'Hike To'] = 'Expert'
whole_table['ability_level'][whole_table['ability_level'] == 'Exp Bowl'] = 'Expert'

/home/karen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/karen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/karen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


Making a list of the useable X columns

In [4]:
X_lst = [column for column in list(whole_table.columns) if whole_table[column].dtype == 'float64']

In [5]:
whole_table['ability_level'].unique()

array(['Expert', 'Intermediate', 'Advanced', 'Low Intermediate', 'Novice',
       'Beginner', 'Glade'], dtype=object)

In [6]:
WP = whole_table[whole_table['resort'] == 'Winter Park']

In [7]:
WP['avg_width_(ft)'] = round((WP['slope_area_(acres)']/WP['slope_length_(ft)'])*43560,2)

/home/karen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
WP.head()

,trail_name,top_elev_(ft),bottom_elev_(ft),vert_rise_(ft),slope_length_(ft),avg_width_(ft),slope_area_(acres),avg_grade_(%),max_grade_(%),ability_level,resort,location
1,Sorensen Park,9150.0,9090.0,60.0,1067.43,166.09,4.07,6.0,8.0,Beginner,Winter Park,CO
2,Turnpike,9460.0,9090.0,370.0,4366.51,51.38,5.15,9.0,15.0,Novice,Winter Park,CO
4,Upper Parkway,9710.0,9615.0,95.0,515.18,233.37,2.76,19.0,20.0,Novice,Winter Park,CO
5,Porcupine,9565.0,9390.0,175.0,1680.76,219.52,8.47,10.0,14.0,Novice,Winter Park,CO
6,Bobcat,9490.0,9400.0,90.0,822.96,80.98,1.53,11.0,9.0,Novice,Winter Park,CO


In [9]:
no_WP = whole_table[whole_table['resort'] != 'Winter Park']

Taking out the Glades, since they're only in Vail, and I'm not sure what "level" they are

In [10]:
no_WP_no_glades = no_WP[no_WP['ability_level'] != 'Glade']

In [11]:
no_WP_no_glades.shape

(902, 12)

In [12]:
y = no_WP_no_glades['ability_level'].values

In [13]:
y_WP = WP['ability_level'].values

In [14]:
X_df = no_WP_no_glades[X_lst]

In [15]:
X = X_df.values

In [16]:
X_WP_df = WP[X_lst]

In [17]:
X_WP = X_WP_df.values

In [18]:
X.shape

(902, 7)

Checking the multicollinearity - top_elev and bottom_elev seem to be an issue

In [19]:
for i in range(X.shape[1]):
    print(variance_inflation_factor(X,i))

230983.310033
211795.340515
685.106585747
8.04823551605
2.07729427927
25.0777058556
30.4859199687


Dropping the columns that seem to be most correlated (and unimportant...)

In [20]:
X_dropped = X_df.drop(['top_elev_(ft)','bottom_elev_(ft)','vert_rise_(ft)'],axis=1)
X_WP_dropped = X_WP_df.drop(['top_elev_(ft)','bottom_elev_(ft)','vert_rise_(ft)'],axis=1)
X_dropped.head()

,slope_length_(ft),slope_area_(acres),avg_grade_(%),max_grade_(%)
0,2404.0,17.6,50.0,69.0
1,839.0,3.7,55.0,66.0
2,1141.0,6.7,42.0,76.0
3,1183.0,3.7,35.0,61.0
4,702.0,3.2,33.0,41.0


In [21]:
X_dropped_mat = X_dropped.values
X_WP_dropped_mat = X_WP_dropped.values

In [22]:
for i in range(X_dropped_mat.shape[1]):
    print(variance_inflation_factor(X_dropped_mat,i))

2.15616742796
1.72004087226
23.3254539864
27.1306624933


In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

Splitting the data and testing on y being the ability level - not a very good accuracy...

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X_dropped_mat,y,test_size=.33,random_state=42)

In [25]:
logmod = LogisticRegression()

In [26]:
logmod.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [27]:
logmod.score(X_test,y_test)

0.54697986577181212

In [28]:
y_pred = logmod.predict(X_test)

Setting y as 'target' and using no_glades - much better accuracy...

In [29]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier

In [30]:
dtc = DecisionTreeClassifier()
dtc.fit (X_train,y_train)
dtc.score(X_test,y_test)

0.71140939597315433

In [31]:
knn = KNeighborsClassifier()
knn.fit(X_train,y_train)
knn.score(X_test,y_test)

0.3825503355704698

In [32]:
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
rfc.score(X_test,y_test)

0.78523489932885904

In [33]:
sgd = SGDClassifier()
sgd.fit(X_train,y_train)
sgd.score(X_test,y_test)

/home/karen/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.18456375838926176

In [34]:
ada = AdaBoostClassifier()
ada.fit(X_train,y_train)
ada.score(X_test,y_test)

0.48322147651006714

In [35]:
gbc = GradientBoostingClassifier()
gbc.fit(X_train,y_train)
gbc.score(X_test,y_test)

0.78859060402684567

In [36]:
y_pred = gbc.predict(X_test)

In [37]:
for i,j in zip(y_pred,y_test):
    if i != j:
        print(i,j)

Intermediate Low Intermediate
Beginner Novice
Low Intermediate Intermediate
Intermediate Low Intermediate
Low Intermediate Intermediate
Advanced Expert
Beginner Intermediate
Intermediate Novice
Intermediate Advanced
Low Intermediate Intermediate
Low Intermediate Intermediate
Novice Low Intermediate
Intermediate Advanced
Novice Beginner
Advanced Intermediate
Intermediate Advanced
Novice Beginner
Novice Low Intermediate
Low Intermediate Intermediate
Advanced Expert
Low Intermediate Intermediate
Intermediate Low Intermediate
Low Intermediate Intermediate
Expert Intermediate
Advanced Expert
Low Intermediate Novice
Low Intermediate Advanced
Low Intermediate Intermediate
Novice Low Intermediate
Novice Low Intermediate
Low Intermediate Intermediate
Intermediate Low Intermediate
Novice Intermediate
Low Intermediate Intermediate
Intermediate Advanced
Novice Low Intermediate
Intermediate Advanced
Advanced Intermediate
Intermediate Low Intermediate
Novice Low Intermediate
Beginner Intermediate
No

In [38]:
y_pred_WP = gbc.predict(X_WP_dropped)

In [40]:
gbc.score(X_WP_dropped_mat,y_WP)

0.53333333333333333

In [49]:
np.where(y_pred_WP != y_WP)[0]

array([  4,   5,   8,   9,  10,  18,  20,  21,  25,  26,  27,  28,  29,
        31,  32,  38,  41,  42,  43,  44,  45,  47,  50,  51,  54,  55,
        56,  60,  64,  69,  70,  71,  72,  74,  77,  78,  81,  82,  84,
        85,  86,  88,  89,  93,  94,  98,  99, 107, 108, 109, 110, 112,
       116, 117, 118, 119, 120, 121, 124, 126, 127, 128, 130, 131, 132,
       133, 134, 135, 136, 137, 138, 140, 141, 142, 143, 144, 145])

In [50]:
y_pred_WP[y_pred_WP != y_WP] + ' - ' + y_WP[y_pred_WP != y_WP] + ' - ' + np.where(y_pred_WP != y_WP)[0].astype(str)

array(['Beginner - Novice - 4', 'Intermediate - Novice - 5',
       'Intermediate - Beginner - 8',
       'Intermediate - Low Intermediate - 9', 'Advanced - Expert - 10',
       'Expert - Advanced - 18', 'Advanced - Expert - 20',
       'Advanced - Expert - 21', 'Intermediate - Low Intermediate - 25',
       'Intermediate - Low Intermediate - 26',
       'Low Intermediate - Novice - 27',
       'Intermediate - Low Intermediate - 28',
       'Novice - Low Intermediate - 29',
       'Intermediate - Low Intermediate - 31', 'Expert - Novice - 32',
       'Novice - Low Intermediate - 38', 'Low Intermediate - Novice - 41',
       'Intermediate - Low Intermediate - 42',
       'Intermediate - Low Intermediate - 43', 'Beginner - Novice - 44',
       'Beginner - Novice - 45', 'Low Intermediate - Novice - 47',
       'Advanced - Expert - 50', 'Novice - Expert - 51',
       'Beginner - Novice - 54', 'Beginner - Novice - 55',
       'Beginner - Novice - 56', 'Novice - Low Intermediate - 60',
     